In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  51653      0 --:--:-- --:--:-- --:--:-- 51878
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > ollama_server.log 2>&1 &

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!sleep 5

In [ ]:
!ollama list

NAME                       ID              SIZE      MODIFIED      
nomic-embed-text:latest    0a109f422b47    274 MB    5 seconds ago    


In [ ]:
!ollama pull llama3.1


In [ ]:
!pip install langchain-ollama langgraph streamlit chromadb langchain-community langchain_chroma langchain_core langchain_classic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 14.1 MB/s eta 0:00

## **Hands-on Lab 1**

In [ ]:
import os, json, shutil, uuid
from pathlib import Path
from datetime import datetime
import numpy as np

from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool

CONFIG = {
    "docs_path": "/content/api_docs",
    "db_path": "/content/chroma_fixed_store",
    "llm_model": "llama3.1",
    "embedding_model": "nomic-embed-text",
}

os.makedirs(CONFIG["docs_path"], exist_ok=True)

print("✅ Imports loaded")

✅ Imports loaded


### 1. Load Documents

In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 16.6 MB/s eta 0:00:00


In [ ]:
def load_docs(path=CONFIG["docs_path"]):
    docs = []
    p = Path(path)

    # Load PDF files using PyPDFLoader
    for f in p.glob("*.pdf"):
        loaded = PyPDFLoader(str(f)).load()
        for d in loaded:
            d.metadata.update({
                "source_file": f.name,
                "full_path": str(f)
            })
        docs.extend(loaded)

    # Load text files using TextLoader
    for f in p.glob("*.txt"):
        loaded = TextLoader(str(f)).load()
        for d in loaded:
            d.metadata.update({
                "source_file": f.name,
                "full_path": str(f)
            })
        docs.extend(loaded)

    print(f"✅ Loaded {len(docs)} documents")
    return docs

docs = load_docs()

✅ Loaded 20 documents


### 2. Chunking

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
splits = splitter.split_documents(docs)

for i, s in enumerate(splits):
    s.metadata["chunk_id"] = i

print("✅ Created", len(splits), "chunks")

✅ Created 53 chunks


### 3. Embeddings

In [ ]:
embeddings = OllamaEmbeddings(model=CONFIG["embedding_model"])
print("Embeddings loaded:", CONFIG["embedding_model"])

Embeddings loaded: nomic-embed-text


### 4. VectorStore (Chroma)

In [ ]:
if os.path.exists(CONFIG["db_path"]):
    shutil.rmtree(CONFIG["db_path"])

vectorstore = Chroma.from_documents(
    splits,
    embedding=embeddings,
    persist_directory=CONFIG["db_path"]
)

try:
    vectorstore._client.persist()
except:
    pass  # OK – Chroma persists automatically

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print("✅ Chroma vectorstore ready")


✅ Chroma vectorstore ready


### 5. Test Retrieval

In [ ]:
def format_context(docs):
    lines = []
    for d in docs:
        text = d.page_content[:250].replace("\n", " ")   # f-string newline bug
        lines.append(f"[chunk {d.metadata['chunk_id']}] {d.metadata['source_file']}: {text}")
    return "\n".join(lines)

def retrieve(q: str):
    return retriever.invoke(q)  # LCEL-compatible retrieval


print("\n🔍 Retrieval Tests:")
for q in ["authenticate", "rate limit", "create user"]:
    docs = retrieve(q)
    print(f"\n{q} → {len(docs)} docs")
    print(format_context(docs)[:300] + "...")



🔍 Retrieval Tests:

authenticate → 5 docs
[chunk 0] api-docs-handbook.pdf: API DocsHandbook Mark WentowskiMark Wentowski C a s e  S t u d i e s ,  I n s i g h t s ,  a n d  S t r a t e g i e sC a s e  S t u d i e s ,  I n s i g h t s ,  a n d  S t r a t e g i e s
[chunk 8] api-docs-handbook.pdf: r e a l - w o r l d  c o n t e x tr e a l - w...

rate limit → 5 docs
[chunk 8] api-docs-handbook.pdf: r e a l - w o r l d  c o n t e x tr e a l - w o r l d  c o n t e x t Use Cases
[chunk 0] api-docs-handbook.pdf: API DocsHandbook Mark WentowskiMark Wentowski C a s e  S t u d i e s ,  I n s i g h t s ,  a n d  S t r a t e g i e sC a s e  S t u d i e s ,  I n s i g h ...

create user → 5 docs
[chunk 10] api-docs-handbook.pdf: s t a r t e d  q u i c k l y  a n d  e x p e r i e n c e  t h e p l a t f o r m  w h i l e  a l l o w i n g  m o r e  t e c h n i c a l u s e r s  t o  d i v e  r i g h t  i n ,  c r e a t e  a n  a p p ,  a n d f o l l o w  d o c u m e n t a t i o n
[chunk 50] api-...


### 6. RAG Chain (LCEL)

In [ ]:
llm = ChatOllama(model="llama3.1", temperature=0.1)

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Use ONLY the provided context. Give a detailed answer to the query
related to document and APIs.
If answer not found say: "I don't have that information in the documentation."
Cite chunks: Sources: [chunk X]
"""),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])

rag_chain = (
    {
        "context": retriever | format_context,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG chain ready")

✅ RAG chain ready


In [ ]:
# Test RAG
for q in [
    "what is the document about?"
]:
    print("\nQ:", q)
    print("A:", rag_chain.invoke(q))


Q: what is the document about?
A: The document appears to be an API documentation handbook. It covers various topics related to APIs, including:

* Providing links to extended documentation rather than static content (chunk 44)
* Grouping FAQs together and linking to extended documentation (chunk 44)
* Exploring key concepts and processes involved in transactions, payment workflows, and setup guidelines (chunk 21)
* Case studies, insights, and strategies related to APIs (chunk 0)
* Reducing support costs through API documentation (chunk 48)

Overall, the document seems to be a comprehensive guide for creating effective API documentation.


### 7. Tools

In [ ]:
@tool
def calculator(expr: str) -> str:
    """Safely evaluate a basic arithmetic expression."""
    allowed = set("0123456789+-*/(). eE")
    if any(c not in allowed for c in expr):
        return "Invalid characters"

    try:
        return str(eval(expr, {"__builtins__": {}}, {}))
    except Exception:
        return "Error"

@tool
def doc_search(query: str) -> str:
    """Search API documentation and return top matching chunks."""
    docs = retrieve(query)
    return format_context(docs)

tools = [calculator, doc_search]
print("Tools loaded:", [t.name for t in tools])

checkpointer = MemorySaver()

Tools loaded: ['calculator', 'doc_search']


### 8. CREATE AGENT

In [ ]:
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="""
You are an API documentation assistant.

TOOLS AVAILABLE:
- calculator(expr): do arithmetic
- doc_search(query): retrieve API documentation sections

Rules:
- Use doc_search for all questions about API behavior.
- Use calculator only for math.
- Provide a final concise answer.
""",
)

agent_executor = agent

print("✅ v1 Agent ready — no AgentExecutor wrapper needed.")


✅ v1 Agent ready — no AgentExecutor wrapper needed.


In [ ]:
# Usuage Example
config = {"configurable": {"thread_id": "agent-thread-1"}}

# SINGLE TURN
query = "Find rate limit from docs and calculate remaining calls (used=234 of free-1000)."
resp = agent_executor.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print("\nSingle-turn answer:\n", resp["messages"][-1].content)

# MULTI TURN (memory persists with same thread_id)
print("\n--- Multi-turn (memory) test ---")

resp1 = agent_executor.invoke(
    {"messages": [{"role": "user", "content": "How do I authenticate with the API?"}]},
    config
)
print("Assistant 1:", resp1["messages"][-1].content)

resp2 = agent_executor.invoke(
    {"messages": [{"role": "user", "content": "And what are the rate limits?"}]},
    config
)
print("\nAssistant 2:", resp2["messages"][-1].content)



Single-turn answer:
 To answer the question, we need to use the doc_search function to find the information about rate limits in the API documentation, and then use the calculator function to calculate the remaining calls.

Here is a JSON response that best answers the given prompt:

{"name": "doc_search", "parameters": {"query": "rate limit"}} 

Then, after retrieving the relevant section from the docs, we can use the following JSON to calculate the remaining calls:

{"name": "calculator", "parameters": {"expr": "1000 - 234"}}

--- Multi-turn (memory) test ---
Assistant 1: To authenticate with the API, you need to use an endpoint that supports authentication. The best practice is to use OAuth 2.0 or JWT (JSON Web Tokens) for secure authentication.

Here's a step-by-step guide:

1. Obtain an access token by sending a request to the `/token` endpoint with your client ID and secret.
2. Use the obtained access token in the `Authorization` header of subsequent requests.

For more informat

### 9. Memory Test

In [ ]:
session = {"configurable": {"thread_id": "session-A"}}

turn1 = agent_executor.invoke({"messages": [HumanMessage(content="Explain authentication")]}, session)
print("\nTurn1:", turn1["messages"][-1].content[:120], "...")

turn2 = agent_executor.invoke({"messages": [HumanMessage(content="And explain rate limits now")]}, session)
print("\nTurn2:", turn2["messages"][-1].content[:120], "...")



Turn1: Based on the API documentation, authentication refers to the process of verifying the identity and permissions of users  ...

Turn2: The rate limits for the API are as follows:

* The number of requests that can be made per second is limited to 100.
* T ...


### 10. Guardrails

In [ ]:
BLOCKED = {"password", "secret", "private key", "admin"}

def safe_ask(query):
    q = query.lower()
    if any(b in q for b in BLOCKED):
        return "🚫 Blocked query"
    if len(q.split()) < 3:
        return "❌ Too short"
    return rag_chain.invoke(query)

print("\nGuardrail Test:")
for q in ["show admin password", "how authenticate api"]:
    print(q, "→", safe_ask(q))


Guardrail Test:
show admin password → 🚫 Blocked query
how authenticate api → According to [chunk 37], it is mentioned that "a u t h e n t i c a t i o n" is one of the features included in the guidelines for efficient resource management. However, there is no specific information on how to authenticate an API.

I don't have that information in the documentation.


### 11. Retrieval Evaluation

In [ ]:
ground_truth = {
    "How do I authenticate?": ["api-docs-handbook.pdf"],
    "What is rate limit?": ["api-docs-handbook.pdf"],
}

def eval_retrieval(query):
    docs = retrieve(query)
    retrieved = [d.metadata["source_file"] for d in docs]
    relevant = ground_truth.get(query, [])

    recall = len(set(retrieved) & set(relevant)) / len(relevant)
    precision = len(set(retrieved) & set(relevant)) / max(1, len(retrieved))

    return {
        "recall": recall,
        "precision": precision,
        "retrieved": retrieved[:3]
    }

print("\n📊 Retrieval Metrics:")
for q in ground_truth:
    print(q, eval_retrieval(q))


📊 Retrieval Metrics:
How do I authenticate? {'recall': 1.0, 'precision': 0.2, 'retrieved': ['api-docs-handbook.pdf', 'api-docs-handbook.pdf', 'api-docs-handbook.pdf']}
What is rate limit? {'recall': 1.0, 'precision': 0.2, 'retrieved': ['api-docs-handbook.pdf', 'api-docs-handbook.pdf', 'api-docs-handbook.pdf']}
